<font color = 'red'>
Content:
    
1. [Load and check data](#1)
3. [Univariate Variable Analysis](#3)
    *          [Categorical Variable ](#4)
    *          [Numerical Variable ](#5)
1. [Outlier Detection](#20)    
1. [Missing Value](#21)     
1. [Correlation matrix](#6)    
1. [Data Manipulation](#7) 
    *           [One-Hot Encoding ](#8)
    *           [Others](#9)
    *           [Data Normalization](#10)
                       
1. [Algorithm Works](#222) 
    *           [Logistic Regression](#12)
    *           [Random Forest](#13)
    *           [Naive Bayes Classifier](#14)
    *           [Stochastic Gradient Descent Classifier](#15) 
    *           [KNN](#16)  
    *           [Decision Tree](#17) 
    *           [Neural Network-Perceptron](#18)
    *           [Gradient Boosting Classifier](#22)
    *           [Xgboost Classifier](#23)
1. [Comparison of algorithms](#19) 
2. [Comparison of Kappa Scores](#24)
3. [Comparison of F1 Scores](#25) 
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id = "1"></a><br>
# Load and check data

In [ ]:
data=pd.read_csv('../input/bank-marketing-dataset/bank.csv')

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.head(10)

In [ ]:
data.describe()


1. Age / Age
2. Job / Job
3. Marital Status / Marital Status
4. Education / Education Level
5. Default / Having a previously broken credit
6. Housing / home loan?
7. Loan / Personal Loan?
8. Contact / Was the customer contacted on his home or mobile phone?
9. Month: Last month of contact
10. Day: The day of the contacted.
11. Duration: Talk time on last call
12. Campaign: The number of contacts reaching the customer during the current campaign (including the last contact)
13. Pdays: The number of days since the previous campaign, if reached (-1 if it was never reached before)
14. Previous: The number of contacts that reached the customer before this campaign
15. Poutcome: Previous campaign success, failure or failure

<a id = "3"></a><br>
# Univariate Variable Analysis
* Categorical Variables:job,marital, default, education,housing,loan,contact,poutcome,mounth,deposit,day
* Numerical Variables: age, campaign,duration, pdays,balance,previous

<a id = "4"></a><br>
## Categorical Variable 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def bar_plot(variable):
    var =data[variable]
    varValue = var.value_counts()
    plt.figure(figsize=(15,3))
    plt.bar(varValue.index, varValue,color=['#00008b','#00e5ee','#cd1076', '#008080','#cd5555','red','blue',])
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
categoryc = ["job","marital","education", "housing", "loan","contact","poutcome","month","deposit"]
for c in categoryc:
    bar_plot(c)
    

<a id = "5"></a><br>
## Numerical Variable

In [ ]:
def plot_hist(variable):
    plt.figure(figsize=(9,6))
    plt.hist(data[variable], bins=40,color='#cd1076')
    plt.xlabel(variable)
    plt.ylabel("frequency")
    plt.title("{} distrubition with hist".format(variable))
    plt.show()

In [ ]:
numericVar = ["age","campaign","duration"]
for n in numericVar:
    plot_hist(n)

In [ ]:
pd.crosstab(data.age,data.deposit).plot(kind="area",figsize=(15,7),color=['#0000ff','#000000' ])
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


### The number of people who are 25 to 40 years old with a time deposit account is high.

In [ ]:
pd.crosstab(data.job,data.deposit).plot(kind="barh",figsize=(15,7),color=['#0000ff','#000000'])
plt.title('Deposit Age Distribution')
plt.xlabel('Frequency')
plt.ylabel('Job')
plt.show()

### In people at the executive level  have more deposit accounts.

<a id = "20"></a><br>
# Outlier Detection

In [ ]:
from collections import Counter
def detect_outliers(data,features):
    outlier_indices = []
    for c in features:
        # 1st quartile
        Q1 = np.percentile(data[c],25)
        # 3rd quartile
        Q3 = np.percentile(data[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = data[(data[c] < Q1 - outlier_step) | (data[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
data.loc[detect_outliers(data,['age',
                               'day','duration','campaign','previous'])]


### I do not include this row

In [ ]:
data = data.drop([3945], axis=0)

<a id = "21"></a><br>
# Missing Value

In [ ]:
data.isnull().sum()

### No missing value..

<a id = "6"></a><br>
# Correlation matrix

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(13,13))         # Sample figsize in inches
sns.heatmap(data.corr(), annot=True, linewidths=.5, ax=ax)

## * Calculated correlation between two variables (r) gets a value between -1 and 1.
* No correlaiton r=0
* Very weak correlation: r<20
* Weak correlation: between 0.20-0.49
* Moderate correlation: between 0.5-0.79
* Strong correlation: between 0.8-0.99
* Perfect correlation: r=1 

### Looking at it, there is a moderate correlation between the *days* and the *previous* ones. (r=0.51)

<a id = "7"></a><br>
# Data Manipulation

### I do not include the *Duration column* in the dataset, as it is unknown data at the time of the prediction.
**duration: Talk Time on Last Call**

In [ ]:
data=data.drop(['duration'],axis=1)

In [ ]:
data.head()

<a id = "8"></a><br>
# One-Hot Encoding 

### **One Hot Encoding means that categorical variables are represented as binary.**

In [ ]:
columns=data.select_dtypes(include=[object]).columns
data=pd.concat([data,pd.get_dummies(data[columns])],axis=1)
data=data.drop(['job','marital','education','default','housing','loan','contact','month','day','poutcome'],axis=1)
data.info()
data.head()

<a id = "9"></a><br>
# Others..

## 1. The ***pdays*** data indicates how many times the customer has been contacted before.

### Updated as follows.

if the **pdays = 0**, it indicates that it has not been contacted before

if the **pdays = 1**, it indicates that it was contacted earlier

In [ ]:
def pdayswork(pdays):
    if(pdays == -1):
        return(0)
    elif(pdays >= 0):
        return(1)
data['pdays2'] = data['pdays'].apply(pdayswork)

### 2. For a single target column

In [ ]:
data=data.drop(['deposit_no', 'deposit_yes'],axis=1)

In [ ]:
def deposit1(deposit):
    if(deposit=='yes'):
        return(1)
    elif(deposit=='no'):
        return(0)
data['depositNew'] = data['deposit'].apply(deposit1)

In [ ]:
data=data.drop(['deposit'],axis=1)

### In this way, our target column, whose data type is object, turned into numerical values. And new target column name is *depositNew*. Also as this is a classification problem, the target column can remain as an object. But I chose to convert it to int data type.

# the current state of our data set.

In [ ]:
data.head()

In [ ]:
data.info()

<a id = "10"></a><br>
# Data Normalization

### StandartScaler, normalizes the data with a standard deviation of 1 with an average of 0.

**The target column is not normalized.**

In [ ]:
from sklearn.preprocessing import StandardScaler
X = data.iloc[:, 0:50]
Y = data.iloc[:, 50]
nd = StandardScaler()
nd.fit(X)
X =nd.transform(X)
print(X)

<a id = "222"></a><br>
# Algorithm Works

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
X = data.iloc[:, 0:50]
Y = data.iloc[:, 50]
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2, random_state = 100)

accuracies = {}
kappaScores= {}
f1scores={}

<a id = "12"></a><br>
# Logistic Regression 

Logistic regression is a predictive linear model that aims to explain the relationship between a dependent binary variable and one or more independent variables. 
The output of Logistic Regression is a number between  *0 and 1* which you can think about as being the probability that a given class is true or not.


In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=101,multi_class='ovr',solver='liblinear',class_weight='balanced',C=0.2)
lr.fit(X_train,y_train)
prediction = lr.predict(X_test)

In [ ]:
print(classification_report(y_test,prediction))
acc = accuracy_score(y_test,prediction)*100
print("Logistic Regression accuracy:",acc)
accuracies['Logistic Regression']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Logistic Regression']=f1


cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Logistic Regression']=cohen_kappa

In [ ]:
score=round(accuracy_score(y_test,prediction),3)
cm= confusion_matrix
cm1=cm(y_test,prediction)
sns.heatmap(cm1, annot=True,fmt=".1f",linewidths=3,square=True, cmap='PuBu',color="#cd1076")
plt.ylabel('actual label')
plt.xlabel('predicted label')
plt.title('accuracy score: {0}'.format(score),size=12)
plt.show()

 <a id = "13"></a><br>
 # Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=12,
                             random_state=50)

clf.fit(X_train,y_train)

prediction = clf.predict(X_test)

In [ ]:
acc = accuracy_score(y_test,prediction)*100
print("Random Forest accuracy:",acc)
accuracies['Random Forest']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Random Forest']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Random Forest']=cohen_kappa

<a id = "14"></a><br>
# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb=GaussianNB()
nb.fit(X_train,y_train)
naiveb=nb.predict(X_test)
prediction= nb.predict(X_test)

In [ ]:
acc = accuracy_score(y_test,prediction)*100
print("Naive Bayes accuracy:",acc)
accuracies['Naive Bayes']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Naive Bayes']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Naive Bayes']=cohen_kappa

 <a id = "15"></a><br>
 # Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd=SGDClassifier(loss='modified_huber',shuffle=True,random_state=100,penalty='l1',alpha=0.004
                  ,max_iter=100,eta0=0.2,learning_rate='optimal')
sgd.fit(X_train,y_train)
prediction=sgd.predict(X_test)

In [ ]:
acc = accuracy_score(y_test,prediction)*100
print("SGD Classifier accuracy:",acc)
accuracies['SGDC']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['SGDC']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['SGDC']=cohen_kappa

<a id = "16"></a><br>
# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn= KNeighborsClassifier(n_neighbors = 4,algorithm='ball_tree')
knn.fit(X_train, y_train)
prediction=knn.predict(X_test)

In [ ]:
acc = accuracy_score(y_test,prediction)*100
print("Knn accuracy:",acc)
accuracies['KNN']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['KNN']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['KNN']=cohen_kappa

<a id = "17"></a><br>
# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree= DecisionTreeClassifier(criterion='gini',max_depth=10,random_state=100,min_samples_leaf=10)
dtree.fit(X_train, y_train)
prediction=dtree.predict(X_test)

In [ ]:
acc = accuracy_score(y_test,prediction)*100
print("Decision Tree accuracy:",acc)
accuracies['Decision Tree']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Decision Tree']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Decision Tree']=cohen_kappa

<a id = "18"></a><br>
# Neural Network - Perceptron

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
pr = Perceptron(alpha=0.07,max_iter=100, random_state=100,penalty='l1')
pr.fit(X_train, y_train)
prediction = pr.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, prediction)*100
print("Perceptron accuracy:",acc)
accuracies['Perceptron']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Perceptron']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Perceptron']=cohen_kappa

<a id = "22"></a><br>
# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.8,
    max_depth=2, random_state=0)
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, prediction)*100
print("Gradient Boosting Classifier accuracy:",acc)
accuracies['Gradient Boosting']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Gradient Boosted']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Gradient Boosting']=cohen_kappa

<a id = "23"></a><br>
# Xgboost Classifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb =XGBClassifier(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.78,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
prediction = xgb.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, prediction)*100
print("Xgboost Classifier accuracy:",acc)
accuracies['Xgboost Classifier']=acc

f1=f1_score(y_test,prediction)*100
print("F1 Score: ",f1)
f1scores['Xgboost Classifier']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Xgboost Classifier']=cohen_kappa

<a id = "19"></a><br>
# Comparison of accuracies

## Accuracy is a metric used to measure the success of a model but is not sufficient by itself.

In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("Accuracy %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(accuracies.keys()), y=list(accuracies.values()), palette=colors)
plt.show()

<a id = "24"></a><br>
# Comparison of Kappa Scores

### Cohen's kappa, (7), symbolized by the lowercase Greek letter, is a powerful statistic useful for testing reliability. Similar to the correlation coefficients, between -1 and +1; where 0 represents the availability that can be expected from random chance, and 1 represents the perfect match between raters.

* 0 indicates no information agreement
* 0.01-0.20 **Slight agreement**
* 0.21-0.40 **Fair agreement**
* 0.41-0.60 **Moderate agreement**
* 0.61-0.80 **Substantial agreement**
* 0.81-1.00 **Almost perfect agreement**

In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("Kappa Score %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(kappaScores.keys()), y=list(kappaScores.values()), palette=colors)
plt.show()

<a id = "25"></a><br>
# Comparison of F1 Scores

 ### The F1 Score value shows us the harmonic mean of the Precision and Recall values.
 
## The main reason for using the F1 Score value instead of Accuracy is not to make an incorrect model selection in non-uniform data sets.

In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("F1 Score %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(f1scores.keys()), y=list(f1scores.values()), palette=colors)
plt.show()


# Please do not forget to write down your positive or negative opinions.